In [8]:
# Loading the dataset
# Using pandas
import pandas as pd
# Reading dataset
dataset = pd.read_table('ratings.dat', header = None, sep = '::', engine = 'python', usecols = [0,1,2], names = ('UserID','MovieID', 'Ratings'))

dataset.head()

# Are there missing values?
dataset.isna().sum()


# Keep in mind that there are “gaps” in numbering of users and items. (Dictionaries? Renumber everything? …)
# What is meant with this??

UserID     0
MovieID    0
Ratings    0
dtype: int64

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold 
import math
from sklearn.metrics import mean_absolute_error as mae
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import mean_squared_error



In [3]:
import matplotlib.pyplot as plt
plt.hist(dataset['Ratings'])
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [5]:
# How many unique Users and Movies there are

num_userID  = len(dataset.UserID.unique())
num_movieID = len(dataset.MovieID.unique())
#(num_userID,num_movieID)




hello
0.0


In [6]:
# Taks 1.1 Naive Approaches
import time

# Takes a long time to run but I think it works sort of correct, have also have to add everything for test 
start = time.time()
n_splits = 5
KF = KFold(n_splits=n_splits, random_state=123, shuffle=True)
GlobalAvergage_RMSE, UserAverage_RMSE, MovieAverage_RMSE, LinReg_RMSE, LinRegInter_RMSE = list(),list(),list(),list(),list()
GlobalAvergage_MAE, UserAverage_MAE, MovieAverage_MAE, LinReg_MAE, LinRegInter_MAE = list(),list(),list(),list(),list()
for train, test in KF.split(dataset):
    #Define X and y in train and test
    X_train, X_test = dataset.iloc[train, 0:2], dataset.iloc[test, 0:2]
    y_train, y_test = dataset.iloc[train, -1], dataset.iloc[test, -1]
    
    # Models
    GlobalAvergage = y_train.mean()
    UserAverage = dataset.iloc[train]
    MovieAverage = dataset.iloc[train]
    
    # Create dataframe for Linear Regression
    UserAverage = UserAverage['Ratings'].groupby(UserAverage['UserID'], group_keys=False).transform('mean') 
    MovieAverage = MovieAverage['Ratings'].groupby(MovieAverage['MovieID'], group_keys=False).transform('mean')
    X_lr = pd.DataFrame(columns = ['UserAverage', 'MovieAverage'])
    X_lr['UserAverage'] = UserAverage
    X_lr['MovieAverage'] = MovieAverage   
    modelLinReg = LR(random_state=123, fit_intercept=False, max_iter= 10000).fit(X_lr,y_train)
    modelLinRegInter = LR(random_state=123, fit_intercept=True, max_iter= 10000).fit(X_lr,y_train)
    
    
    # Predictions 
    pred_GlobalAvergage = [GlobalAvergage] * len(X_train) 
    pred_UserAverage = UserAverage
    pred_MovieAverage = MovieAverage
    pred_LinReg = modelLinReg.predict(X_lr)
    pred_LinRegInter = modelLinRegInter.predict(X_lr)

    #RMSE's 
    RMSE_GlobalAvergage = math.sqrt(mean_squared_error(pred_GlobalAvergage,y_train))
    RMSE_UserAverage = math.sqrt(mean_squared_error(pred_UserAverage,y_train))
    RMSE_MovieAverage = math.sqrt(mean_squared_error(pred_MovieAverage,y_train))
    RMSE_LinReg = math.sqrt(mean_squared_error(pred_LinReg,y_train))
    RMSE_LinRegInter = math.sqrt(mean_squared_error(pred_LinRegInter,y_train))
    
    GlobalAvergage_RMSE.append(RMSE_GlobalAvergage)
    UserAverage_RMSE.append(RMSE_UserAverage)
    MovieAverage_RMSE.append(RMSE_MovieAverage)
    LinReg_RMSE.append(RMSE_LinReg)
    LinRegInter_RMSE.append(RMSE_LinRegInter)
    
    # MAE's 
    MAE_GlobalAvergage = mae(pred_GlobalAvergage, y_train)
    MAE_UserAverage = mae(pred_UserAverage, y_train)
    MAE_MovieAverage = mae(pred_MovieAverage, y_train)
    MAE_LinReg = mae(pred_LinReg, y_train)
    MAE_LinRegInter = mae(pred_LinRegInter, y_train)
    
    GlobalAvergage_MAE.append(MAE_GlobalAvergage)
    UserAverage_MAE.append(MAE_UserAverage)
    MovieAverage_MAE.append(MAE_MovieAverage)
    LinReg_MAE.append(MAE_LinReg)
    LinRegInter_MAE.append(MAE_LinRegInter)
    
end = time.time()
print(end - start)

135.6486690044403


In [6]:
Average_RMSE_GlobalAvergage = np.mean(GlobalAvergage_RMSE)
Average_MAE_GlobalAvergage =  np.mean(GlobalAvergage_MAE)

Average_RMSE_UserAverage = np.mean(UserAverage_RMSE)
Average_MAE_UserAverage =  np.mean(UserAverage_MAE)

Average_RMSE_MovieAverage = np.mean(MovieAverage_RMSE)
Average_MAE_MovieAverage = np.mean(MovieAverage_MAE)

Average_RMSE_LinReg = np.mean(LinReg_RMSE)
Average_MAE_LinReg = np.mean(LinReg_MAE)

Average_RMSE_LinRegInter = np.mean(LinRegInter_RMSE)
Average_MAE_LinRegInter = np.mean(LinRegInter_MAE)

[0.6963733820564957, 0.696703313183373, 0.6956247883254371, 0.6961271834504548, 0.6953664730406615]


## Task 1.2


In [10]:
dataset_task2 = dataset.pivot(
    index='UserID',
    columns='MovieID',
    values='Ratings'
)

In [5]:
def Preprocessing(dataset):
    dataset1 = dataset.sub(dataset.mean(axis = 1), axis=0)
    dataset_end = dataset1.sub(dataset1.mean(axis = 0), axis = 1) #not needed? normalizing among movie axis
    return dataset1
Preprocessing(dataset_task2)

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,0.811321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,-1.146465,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,NaN,NaN,NaN,-1.302928,NaN,-0.302928,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
def Initialization(dataset):
    d = 5
    n = dataset.shape[0]
    m = dataset.shape[1]
    a = dataset.stack().mean()
    U = np.empty([n,d])
    V = np.empty([d,m])
    U.fill(math.sqrt(a/d))
    V.fill(math.sqrt(a/d))
    return U,Vb



(array([[0.8442937, 0.8442937, 0.8442937, 0.8442937, 0.8442937],
        [0.8442937, 0.8442937, 0.8442937, 0.8442937, 0.8442937],
        [0.8442937, 0.8442937, 0.8442937, 0.8442937, 0.8442937],
        [0.8442937, 0.8442937, 0.8442937, 0.8442937, 0.8442937],
        [0.8442937, 0.8442937, 0.8442937, 0.8442937, 0.8442937]]),
 array([[0.8442937, 0.8442937, 0.8442937, ..., 0.8442937, 0.8442937,
         0.8442937],
        [0.8442937, 0.8442937, 0.8442937, ..., 0.8442937, 0.8442937,
         0.8442937],
        [0.8442937, 0.8442937, 0.8442937, ..., 0.8442937, 0.8442937,
         0.8442937],
        [0.8442937, 0.8442937, 0.8442937, ..., 0.8442937, 0.8442937,
         0.8442937],
        [0.8442937, 0.8442937, 0.8442937, ..., 0.8442937, 0.8442937,
         0.8442937]]))

In [5]:
dataset1 = dataset.sub(dataset_task2.mean(axis = 1), axis=0)
dataset_end = dataset1.sub(dataset1.mean(axis = 0), axis = 1) #not needed? normalizing among movie axis

d = 5
n = dataset_end.shape[0]
m = dataset_end.shape[1]
a = dataset_end.stack().mean()
U = np.empty([n,d])
V = np.random.uniform(low = -2, high = 2, size = (d, m)) # random numbers to increase chance of reachine global minimum
U = np.random.randn(n, d)

print(V.shape, U.shape)

(5, 3) (1000209, 5)


In [11]:
dataset_task2 = dataset.pivot(
    index='UserID',
    columns='MovieID',
    values='Ratings'
)

dataset1 = dataset_task2.sub(dataset_task2.mean(axis = 1), axis=0)
dataset_end = dataset1.sub(dataset1.mean(axis = 0), axis = 1)


#print(dataset.sort_values(['UserID', 'MovieID']),df_test.sort_values(by=['UserID', 'MovieID']).dropna())
#pd.melt(dataset_task2, id_vars='')

In [33]:
import time
start = time.time()
dataset1 = dataset_task2.sub(dataset_task2.mean(axis = 1), axis=0)
dataset_end = dataset1.sub(dataset1.mean(axis = 0), axis = 1)

M = dataset_end.to_numpy()
d = 2
n = dataset_end.shape[0]
m = dataset_end.shape[1]
a = dataset_end.stack().mean()
U = np.empty([n,d])
V = np.random.uniform(low = -2, high = 2, size = (d, m)) # random numbers to increase chance of reachine global minimum
U = np.random.randn(n, d)
UV = np.matmul(U,V)


def create_permutation():
    
    n=np.random.permutation(range(6040))
    m=np.random.permutation(range(3706))
    return n,m


rows_not_blank=[]                     #  for every row i store the column indices j that aren't blank
columns_not_blank=[]                  #  for every row j store the row indices i that aren't blank

for i in range(M.shape[0]):
    rows_not_blank.append(np.where(np.isfinite(M[i,:])))
    
for i in range(M.shape[1]):
    columns_not_blank.append((np.argwhere(np.isfinite(M[:,i]))))

all_not_blank=np.where(np.isfinite(M))   # store indeces of non_blanks and only run loops over these

######################################################################################################################

epoch_counter = 0
new_epoch_rmse = 1000
while epoch_counter<2:
    rmse=new_epoch_rmse
    epoch_counter+=1
    
    
    perms_n , perms_m =create_permutation()



    for count in range(6040):
        r=perms_n[count]
        #Update V values 
        if count<3706:
    
            s=perms_m[count]
            #first row
            if not len(columns_not_blank[s])==0:  # taking care of cases where movie is not present in the training set
        
                element=V[0,s]
                denominator=0
                numerator=0
                new_element=0
        
                for i in columns_not_blank[s]:
                 
            
                    numerator+=(U[i,0]*(M[i,s] -(U[i,1]*V[1,s])))
                    denominator+=(U[i,0]**2)
                   
                new_element=numerator/denominator
        
                V[0,s]=(new_element+element)/2      # set value to half way between optimal and original
            
            #second row
            if not len(columns_not_blank[s])==0:
            
        
                element=V[1,s]
                denominator=0
                numerator=0
                new_element=0
        
                for i in columns_not_blank[s]:
                
                
                    numerator+=(U[i,1]*(M[i,s] -(U[i,0]*V[0,s])))
                    denominator+=(U[i,1]**2)
           
                new_element=numerator/denominator
        
                V[1,s]=(new_element+element)/2   # set value to half way between optimal and original

        #Update U values
        
        #first column
        element=U[r,0]
        denominator=0
        numerator=0
        new_element=0
        
        for j in rows_not_blank[r]:
            numerator+=(V[0,j]*(M[r,j] -(U[r,1]*V[1,j])))
            denominator+=(V[0,j]**2)
        
        new_element=numerator/denominator
        U[r,0]=(new_element+element)/2       # set value to half way between optimal and original
        
        #second column
        element=U[r,1]
        denominator=0
        numerator=0
        new_element=0
        
        for j in rows_not_blank[r]:
            numerator+=(V[1,j]*(M[r,j] -(U[r,0]*V[0,j])))
            denominator+=(V[1,j]**2)
        
        new_element=numerator/denominator
        U[r,1]=new_element      # set value to half way between optimal and original
    P=np.matmul(U,V)  # updated P matrix
    P[P>5]=5
    P[P<1]=1
    abs_errors=0
    squared_errors=0
    for i,j in all_not_blank_positions:
        squared_errors+=(M[i,j]-P[i,j])**2   
        abs_errors+=np.abs((M[i,j]-P[i,j]))
        
    mae_val=math.sqrt(abs_errors/len(all_not_blank_positions))
    new_epoch_rmse=math.sqrt(squared_errors/len(all_not_blank_positions))
end = time.time()
print(end - start)


ValueError: setting an array element with a sequence.

In [7]:
#initialize vars
dataset1 = dataset_task2.sub(dataset_task2.mean(axis = 1), axis=0)
dataset_end = dataset1.sub(dataset1.mean(axis = 0), axis = 1)

M = dataset_end.to_numpy()
d = 2
n = dataset_end.shape[0]
m = dataset_end.shape[1]
a = dataset_end.stack().mean()
U = np.empty([n,d])
V = np.random.uniform(low = -2, high = 2, size = (d, m)) # random numbers to increase chance of reachine global minimum
U = np.random.randn(n, d)
uv = np.matmul(U,V)


#make train and test sets



for 
#U
for r in range(U.shape[0]):
    numerator = 0
    denominator = 0

    for s in range(d):
        U_rk = U[r,:]
        U_rk = np.delete(U_rk, s, 0)
        V_kj = np.delete(V, s, 0)
        V_sj = V[s,:]
        P = np.matmul(U_rk, V_kj)
        m_rj = M[r,:]
        numerator = V_sj * (m_rj - P)
        numerator = np.nansum(numerator * (np.isfinite(m_rj)))
        denominator = V_sj ** 2
        denominator = np.nansum(denominator * (np.isfinite(m_rj)))
        new_var = numerator/denominator
        U[r,s] = new_var

for r in range(d):
    for s in range(V.shape[1]):
        V_ks = V[:,s]
        V_ks = np.delete(V_ks, r, 0)
        U_ik = np.delete(U, r, 1)
        U_ir = U[:,r]
        P = np.matmul(U_ik, V_ks)
        m_is = M[:,s]
        numerator = U_ir * (m_is - P)
        numerator = np.nansum(numerator * (np.isfinite(m_is)))
        denominator = U_ir ** 2
        denominator = np.nansum(denominator * (np.isfinite(m_is)))
        newvar = numerator/denominator
        V[r,s] = newvar

#rewrite this part
uv = np.matmul(U,V)
dif = uv - M
dif_sqr = dif ** 2
dif_sqr_0s = np.nan_to_num(dif_sqr)
dif_sqr_total= np.sum( dif_sqr_0s ,axis=0)
sumz = dif_sqr_total.sum()
non_0_count_sqr = np.count_nonzero( dif_sqr_0s )
RME = sumz/ non_0_count_sqr
rme_list=[RME]
print('RMSE=',RME)

RMSE= 3.559348380929595
RMSE= 0.8287853060409066


In [128]:
import time
start = time.time()
n_splits = 5
KF = KFold(n_splits=5, random_state=123, shuffle=True)
rme_list = list()
RME = 1000
halt = True
for Train, Test in KF.split(dataset):
    dataset_2_Train, dataset_2_Test = dataset.loc[Train], dataset.loc[Test]
    dataset_split = dataset_2_Train.pivot(
    index='UserID',
    columns='MovieID',
    values='Ratings'
    )
    M = dataset_split.to_numpy()
    d = 2
    n = dataset_split.shape[0]
    m = dataset_split.shape[1]
    a = dataset_split.stack().mean()
    U = np.empty([n,d])
    V = np.random.randn(d, m) # random numbers to increase chance of reachine global minimum
    U = np.random.randn(n, d)
    uv = np.matmul(U,V)
    
    for i in range (5):
        RME_old = RME
        for r in range(U.shape[0]):
            numerator = 0
            denominator = 0

            for s in range(d):
                U_rk = U[r,:]
                U_rk = np.delete(U_rk, s, 0)
                V_kj = np.delete(V, s, 0)
                V_sj = V[s,:]
                P = np.matmul(U_rk, V_kj)
                m_rj = M[r,:]
                numerator = V_sj * (m_rj - P)
                numerator = np.nansum(numerator * (np.isfinite(m_rj)))
                denominator = V_sj ** 2
                denominator = np.nansum(denominator * (np.isfinite(m_rj)))
                new_var = numerator/denominator
                U[r,s] = new_var

        for r in range(d):
            for s in range(V.shape[1]):
                V_ks = V[:,s]
                V_ks = np.delete(V_ks, r, 0)
                U_ik = np.delete(U, r, 1)
                U_ir = U[:,r]
                P = np.matmul(U_ik, V_ks)
                m_is = M[:,s]
                numerator = U_ir * (m_is - P)
                numerator = np.nansum(numerator * (np.isfinite(m_is)))
                denominator = U_ir ** 2
                denominator = np.nansum(denominator * (np.isfinite(m_is)))
                newvar = numerator/denominator
                V[r,s] = newvar
        
        uv = np.matmul(U,V)
        dif = uv - M
        dif_sqr = dif ** 2
        dif_sqr_0s = np.nan_to_num(dif_sqr)
        dif_sqr_total= np.sum( dif_sqr_0s ,axis=0)
        sumz = dif_sqr_total.sum()
        non_0_count_sqr = np.count_nonzero( dif_sqr_0s )
        RME = sumz/ non_0_count_sqr
        rme_list.append(RME)
        
        halt = np.abs(RME - RME_old) > 0.01
    print('RMSE=',RME)
print('Avg=', np.mean(rme_list))
ende = time.time()
print((ende - start)/60)

RMSE= 0.8005336877196926
RMSE= 0.8109030108624024
RMSE= 0.8095401137866102
RMSE= 0.7947833500448447
RMSE= 0.7706316872647513
Avg= 3.174475248227738
1.0618033011754353


In [129]:
rme_list

[12.300651503945806,
 1.1170971494356627,
 0.8340462464218202,
 0.80979466234258,
 0.8005336877196926,
 13.305991518195862,
 2.3228430052210407,
 0.8894124129365891,
 0.8251045384991118,
 0.8109030108624024,
 10.348599581710483,
 0.9445045614892835,
 0.8292923701967514,
 0.8144583526124056,
 0.8095401137866102,
 11.632694143966006,
 1.2367125257570157,
 0.8446118379894362,
 0.8088226600360423,
 0.7947833500448447,
 12.72161740984856,
 1.1805079976278936,
 0.8211282520764194,
 0.7875986257063804,
 0.7706316872647513]

In [117]:
np.abs(RME - new_rmse)

0.0